In [ ]:
import json
import sagemaker
import boto3
import os
from pprint import pprint
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.serverless import ServerlessInferenceConfig

"""
Be sure to set these environment variables:
os.environ['AWS_PROFILE'] = your-sagemaker-aws-profile in /aws/config
os.environ['HF_TOKEN'] = your-hugging-face-access-token
"""

MODEL_NAME = 'meta-llama/Llama-3.2-1B-Instruct'

role = sagemaker.get_execution_role()

hub = {
	'HF_MODEL_ID': MODEL_NAME,
	'SM_NUM_GPUS': json.dumps(1),
	'HUGGING_FACE_HUB_TOKEN': os.getenv('HF_TOKEN')
	}

print(f"Role: {role}")


[12/17/24 10:43:35] WARNING  Couldn't call 'get_role' to get Role ARN from role name sagemaker-user ]8;id=25179;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=752099;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py#5902\5902]8;;\
                             to get Role path.                                                                     

Role: arn:aws:iam::993678079972:role/sagemaker-user


In [ ]:
# Delete previous endpoint
sagemaker_client = boto3.client("sagemaker", region_name='us-east-1')
sagemaker_client.delete_endpoint(EndpointName="rag-lite-endpoint")
sagemaker_client.delete_endpoint_config(EndpointConfigName="rag-lite-endpoint")

{'ResponseMetadata': {'RequestId': 'dd726731-5df1-4bc1-9d40-369c56daefdf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dd726731-5df1-4bc1-9d40-369c56daefdf',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 17 Dec 2024 16:55:32 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [ ]:
# Create model for serverless inference

huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface", version="2.3.1"),
	env=hub,
	role=role,
)

serverless_config = ServerlessInferenceConfig(
	memory_size_in_mb=6144,
	max_concurrency=1,
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    serverless_config=serverless_config,
	endpoint_name="rag-lite-endpoint"
)


[12/17/24 10:43:38] INFO     Defaulting to only available Python version: py311                   ]8;id=748017;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=449027;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/image_uris.py#603\603]8;;\

                    INFO     Defaulting to only supported image scope: gpu.                       ]8;id=457336;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=321316;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/image_uris.py#527\527]8;;\

                    INFO     Creating model with name:                                              ]8;id=673752;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=681400;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py#4025\4025]8;;\
                             huggingface-pytorch-tgi-inference-2024-12-17-15-43-38-812                             

[12/17/24 10:43:39] INFO     Creating endpoint-config with name llama-1b-endpoint                   ]8;id=240037;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=350157;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py#5820\5820]8;;\

                    INFO     Creating endpoint with name llama-1b-endpoint                          ]8;id=863882;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=533578;file:///Users/benjaminbasseri/miniforge3/envs/sagemaker/lib/python3.12/site-packages/sagemaker/session.py#4642\4642]8;;\

-----------!

In [21]:
prompt = """
References:
\"\"\"
topology is the branch of topology dealing with the basic set-theoretic definitions and constructions used in topology.<sup id=\"cite_ref-11\" class=\"reference\"><a href=\"#cite_note-11\"><span class=\"cite-bracket\">&#91;</span>11<span class=\"cite-bracket\">&#93;</span></a></sup><sup id=\"cite_ref-12\" class=\"reference\"><a href=\"#cite_note-12\"><span class=\"cite-bracket\">&#91;</span>12<span class=\"cite-bracket\">&#93;</span></a></sup> It is the foundation of most other branches of topology, including differential topology, geometric topology, and algebraic topology. Another name for general topology is point-set topology.</p><p>The basic object of study is <a href=\"/wiki/Topological_space\" title=\"Topological space\">topological spaces</a>, which are sets equipped with a <a href=\"/wiki/Topology_(structure)\" class=\"mw-redirect\" title=\"Topology (structure)\">topology</a>, that is, a family of <a href=\"/wiki/Subset\" title=\"Subset\">subsets</a>, called <i>open sets</i>, which is <a
###
\"class=\"reference\"><a href=\"#cite_note-20\"><span class=\"cite-bracket\">&#91;</span>20<span class=\"cite-bracket\">&#93;</span></a></sup></p><div class=\"mw-heading mw-heading3\"><h3 id=\"Physics\">Physics</h3><span class=\"mw-editsection\"><span class=\"mw-editsection-bracket\">[</span><a href=\"/w/index.php?title=Topology&amp;action=edit&amp;section=16\" title=\"Edit section: Physics\"><span>edit</span></a><span class=\"mw-editsection-bracket\">]</span></span></div><p>Topology is relevant to physics in areas such as <a href=\"/wiki/Condensed_matter_physics\" title=\"Condensed matter physics\">condensed matter physics</a>,<sup id=\"cite_ref-21\" class=\"reference\"><a href=\"#cite_note-21\"><span class=\"cite-bracket\">&#91;</span>21<span class=\"cite-bracket\">&#93;</span></a></sup> <a href=\"/wiki/Quantum_field_theory\" title=\"Quantum field theory\">quantum field theory</a> and <a href=\"/wiki/Physical_cosmology\" title=\"Physical cosmology\">physical cosmology</a>.</p><p>The topological dependence of mechanical\
\"\"\"

Does topology have any real-world applications?
"""

In [49]:

response = predictor.predict(
    {
        "inputs": "Who were Lady Abney's friends?",
        "parameters": {
            "max_new_tokens": 500,
            "temperature": 0.2,
            # "top_k": 50,
            # "top_p": 0.95,
        },
    }
)
pprint(response[0]['generated_text'])

("Who were Lady Abney's friends? \n"
 'I am trying to research the history of Lady Abney, a woman who lived in the '
 '18th century.  I am looking for information on who her friends were.  I am '
 'trying to research the history of the family of Abney, who were English '
 "aristocrats.  I am looking for information on the family's history, their "
 'social status, and their relationships with other families of the time.  I '
 'am also looking for information on the social and cultural norms of the 18th '
 "century.  I am trying to research the history of the family's estate, which "
 'was located in the countryside.  I am looking for information on the '
 "family's estate, including its layout, its furnishings, and its history of "
 "ownership.  I am also looking for information on the family's social and "
 'cultural norms, including their role in society and their relationships with '
 'other families of the time.  I am trying to research the history of the '
 "family's estate, inclu

In [ ]:
informed_prompt = """\"class=\\\"reference\\\"><a href=\\\"#cite_note-20\\\"><span class=\\\"cite-bracket\\\">&#91;</span>20<span class=\\\"cite-bracket\\\">&#93;</span></a></sup></p><div class=\\\"mw-heading mw-heading3\\\"><h3 id=\\\"Physics\\\">Physics</h3><span class=\\\"mw-editsection\\\"><span class=\\\"mw-editsection-bracket\\\">[</span><a href=\\\"/w/index.php?title=Topology&amp;action=edit&amp;section=16\\\" title=\\\"Edit section: Physics\\\"><span>edit</span></a><span class=\\\"mw-editsection-bracket\\\">]</span></span></div><p>Topology is relevant to physics in areas such as <a href=\\\"/wiki/Condensed_matter_physics\\\" title=\\\"Condensed matter physics\\\">condensed matter physics</a>,<sup id=\\\"cite_ref-21\\\" class=\\\"reference\\\"><a href=\\\"#cite_note-21\\\"><span class=\\\"cite-bracket\\\">&#91;</span>21<span class=\\\"cite-bracket\\\">&#93;</span></a></sup> <a href=\\\"/wiki/Quantum_field_theory\\\" title=\\\"Quantum field theory\\\">quantum field theory</a> and <a href=\\\"/wiki/Physical_cosmology\\\" title=\\\"Physical cosmology\\\">physical cosmology</a>.</p><p>The topological dependence of mechanical\"

\"topology is the branch of topology dealing with the basic set-theoretic definitions and constructions used in topology.<sup id=\\\"cite_ref-11\\\" class=\\\"reference\\\"><a href=\\\"#cite_note-11\\\"><span class=\\\"cite-bracket\\\">&#91;</span>11<span class=\\\"cite-bracket\\\">&#93;</span></a></sup><sup id=\\\"cite_ref-12\\\" class=\\\"reference\\\"><a href=\\\"#cite_note-12\\\"><span class=\\\"cite-bracket\\\">&#91;</span>12<span class=\\\"cite-bracket\\\">&#93;</span></a></sup> It is the foundation of most other branches of topology, including differential topology, geometric topology, and algebraic topology. Another name for general topology is point-set topology.</p><p>The basic object of study is <a href=\\\"/wiki/Topological_space\\\" title=\\\"Topological space\\\">topological spaces</a>, which are sets equipped with a <a href=\\\"/wiki/Topology_(structure)\\\" class=\\\"mw-redirect\\\" title=\\\"Topology (structure)\\\">topology</a>, that is, a family of <a href=\\\"/wiki/Subset\\\" title=\\\"Subset\\\">subsets</a>, called <i>open sets</i>, which is <a

Does topology have any real-world applications?"""

response = predictor.predict(
    {
        "inputs": informed_prompt,
        "parameters": {
            "max_new_tokens": 1000,
            "temperature": 0.2,
        },
    }
)
pprint(response[0]['generated_text'])

('"class=\\"reference\\"><a href=\\"#cite_note-20\\"><span '
 'class=\\"cite-bracket\\">&#91;</span>20<span '
 'class=\\"cite-bracket\\">&#93;</span></a></sup></p><div class=\\"mw-heading '
 'mw-heading3\\"><h3 id=\\"Physics\\">Physics</h3><span '
 'class=\\"mw-editsection\\"><span '
 'class=\\"mw-editsection-bracket\\">[</span><a '
 'href=\\"/w/index.php?title=Topology&amp;action=edit&amp;section=16\\" '
 'title=\\"Edit section: Physics\\"><span>edit</span></a><span '
 'class=\\"mw-editsection-bracket\\">]</span></span></div><p>Topology is '
 'relevant to physics in areas such as <a '
 'href=\\"/wiki/Condensed_matter_physics\\" title=\\"Condensed matter '
 'physics\\">condensed matter physics</a>,<sup id=\\"cite_ref-21\\" '
 'class=\\"reference\\"><a href=\\"#cite_note-21\\"><span '
 'class=\\"cite-bracket\\">&#91;</span>21<span '
 'class=\\"cite-bracket\\">&#93;</span></a></sup> <a '
 'href=\\"/wiki/Quantum_field_theory\\" title=\\"Quantum field '
 'theory\\">quantum field theory</a>